In [2]:
import tensorflow as tf

In [3]:
from tensorflow.python.framework import function

In [4]:
a =[[1,2], [3,4]]
b = tf.cast(a, dtype = tf.float32)

In [5]:
tf.reduce_sum(tf.reduce_sum(b, axis= 1), axis = 0)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [23]:
@tf.custom_gradient
def bar(x, y):
  def grad(upstream):
    dz_dx = y
    dz_dy = x
    print('upstream')
    print(upstream)
    return upstream * dz_dx, upstream * dz_dy
  z1 = x * y
  return z1, grad
def loss(x,y,z): 
    
    dump = z*x*bar(x,y)
    return dump
x = tf.Variable(5.0, dtype=tf.float32)
y = tf.Variable(3.0, dtype=tf.float32)
z = tf.constant(0.0, dtype= tf.float32)

with tf.GradientTape() as tape:
  tape.watch(x)
  tape.watch(y)
  z1 = loss(x, y,z)
  g = tape.gradient(z1, [x,y])
print(g)

upstream
tf.Tensor(0.0, shape=(), dtype=float32)
[<tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>]


In [22]:
x = tf.cast([[0.0+2.0j], [0.0+4.0j]], dtype = tf.complex64)
y= tf.cast([[0.0+2.0j], [0.0+4.0j]], dtype = tf.complex64)
tf.matmul(tf.transpose(tf.math.conj(y)), x)

<tf.Tensor: shape=(1, 1), dtype=complex64, numpy=array([[20.+0.j]], dtype=complex64)>

In [17]:
tf.transpoy

<tf.Tensor: shape=(2,), dtype=complex64, numpy=array([1.+2.j, 2.+4.j], dtype=complex64)>

In [15]:
f = x.read_value()
f

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [18]:
tf.type(f)

AttributeError: module 'tensorflow' has no attribute 'type'

In [17]:
tf.complex(0, 1)

TypeError: The `real` and `imag` components have incorrect types: int32 int32. They must be consistent, and one of [tf.float32, tf.float64]

In [14]:
x

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>

In [6]:
@tf.function.Defun(tf.float32)
def quad_grad(x): 
    return 2*x
@tf.function.Defun(tf.float32, grad_func=quad_grad)
def quad(x): 
    return x**2


AttributeError: 'function' object has no attribute 'Defun'

In [7]:
quad(2)

AttributeError: Tensor.op is meaningless when eager execution is enabled.

In [2]:
class Model(tf.Module):
  def __init__(self):
    self.dataset = None

  @tf.function
  def __call__(self):
    print("trace") # This will print once: only traced once
    if self.dataset is None:
      self.dataset = tf.data.Dataset.from_tensors([1, 2, 3])
    it = iter(self.dataset)
    return next(it)

m = Model()
m()

trace


<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3])>

In [3]:
class Model(tf.Module):
  def __init__(self):
    self.v = None
    self.dataset = None

  @tf.function
  def __call__(self):
    print("trace") # This will print twice because the python body is run twice
    if self.v is None:
      self.v = tf.Variable(0)
    if self.dataset is None:
      self.dataset = tf.data.Dataset.from_tensors([1, 2, 3])
   # it = iter(self.dataset)
    it = self.dataset[0]
    return [self.v, it]
    #return None

m = Model()
try:
  m()
except TypeError as e:
  print(e) # <tf.Tensor ...> is out of scope and cannot be used here.

trace
in user code:

    File "<ipython-input-3-b565cf74f86e>", line 14, in __call__  *
        it = self.dataset[0]

    TypeError: 'TensorDataset' object is not subscriptable



# Testing Operations

In [8]:
t = tf.constant(2.0)
u = tf.constant(3.0)
tu = tf.multiply(t, u)
tu

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [13]:
t = tf.constant(2.0)
u = tf.constant(3.0)
tu = tf.multiply(t, u)

sess = tf.compat.v1.Session()
with sess.as_default():
    sess.run(tu)  # evaluates both tensors in a single step

RuntimeError: The Session graph is empty. Add operations to the graph before calling run().

In [15]:
 # Launch the graph in a session.
 with tf.compat.v1.Session() as ses:

     # Build a graph.
     a = tf.constant(5.0)
     b = tf.constant(6.0)
     c = a * b

     # Evaluate the tensor `c`.
     print(ses.run(c))

30.0


In [90]:
class test0():
    def __init__(self):
        self.a = tf.constant(0.0)
        self.b = tf.constant(2.0)
    
    #@tf.function
    def mull(self):
        self.c = tf.multiply(self.a, self.b)
        return self.c
    
  # Launch the graph in a session.
with tf.compat.v1.Session() as ses:

    # Build a graph.
    t = test0()
    d = t.mull()
    # Evaluate the tensor `c`.
    print(ses.run(t.c))
    
    t.a = tf.constant(2.0)
    t.b = tf.constant(1.0)
    print(ses.run(t.c))
    print(t.c)
    
    e = t.mull()
    print(ses.run(t.c))

0.0
0.0
Tensor("Mul_8:0", shape=(), dtype=float32)
2.0


In [67]:
class test():
    def __init__(self):
        self.a = None
        self.b = None
    
    @tf.function
    def mull(self,a , b):
        self.a = tf.constant(a)
        self.b = tf.constant(b)
        self.c = self.a * self.b
        return self.c

In [69]:
t = test()
t.mull(1,2)

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [70]:
t.a = tf.constant(4.0)
t.b = tf.constant(5.0)

In [72]:
t.mull(4, 5)

<tf.Tensor: shape=(), dtype=int32, numpy=20>

In [40]:
t.a

<tf.Tensor: shape=(), dtype=float32, numpy=5.0>

In [36]:
print(t.c)

Tensor("mul:0", shape=(), dtype=float32)


In [73]:
@tf.function
def f(a,b):
    print("a:", a, " , b: ",b )
    return a + b


In [84]:
a = tf.constant(1)
b = tf.constant(2)
f(a,b)

a: Tensor("a:0", shape=(), dtype=int32)  , b:  Tensor("b:0", shape=(), dtype=int32)


<tf.Tensor: shape=(), dtype=int32, numpy=3>

In [85]:
a = tf.constant(4)
b = tf.constant(5)
f(a,b)

<tf.Tensor: shape=(), dtype=int32, numpy=9>

In [76]:
a = 2
b = tf.constant(3)
f(a,b)

a: 2  , b:  Tensor("b:0", shape=(), dtype=int32)


<tf.Tensor: shape=(), dtype=int32, numpy=5>

# Norm

In [10]:
a = tf.constant(5.0)
tf.nn.l2_loss(a)

<tf.Tensor: shape=(), dtype=float32, numpy=12.5>

In [13]:
a = [[1.0, 0.0],[0.0, 5.0]]
a_t = tf.convert_to_tensor(a)
a_t

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 0.],
       [0., 5.]], dtype=float32)>

In [14]:
tf.nn.l2_loss(a_t)

<tf.Tensor: shape=(), dtype=float32, numpy=13.0>

In [15]:
a = [1,2]
b = [2,4]
a.extend(b)

In [16]:
a

[1, 2, 2, 4]